<a href="https://colab.research.google.com/github/ucaokylong/IMDB_classification/blob/main/PretrainedBERT_IMDB_KyLong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84125825/84125825 [==============================] - 2s 0us/step


['test', 'imdbEr.txt', 'README', 'train', 'imdb.vocab']

In [3]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['pos',
 'unsupBow.feat',
 'urls_neg.txt',
 'unsup',
 'urls_unsup.txt',
 'labeledBow.feat',
 'urls_pos.txt',
 'neg']

In [4]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [6]:
batch_size = 32
seed = 12345
train_ds = tf.keras.utils.text_dataset_from_directory(
                            'aclImdb/train', batch_size=batch_size, 
                            validation_split=0.2,
                            subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
                            'aclImdb/train', batch_size=batch_size, 
                            validation_split=0.2,
                            subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [7]:
for text_batch, label_batch in train_ds:
    print(label_batch[0].numpy())
    print(text_batch[0].numpy().decode())
    break

0
The movie is basically a boring string of appalling clichés which do not offer a real cross-cultural insight. The Middle Eastern leg of the journey is described in a particularly irritating way: there obviously are mud brick villages, dirt tracks in the middle of the desert, women clad in black robes and belly dancers. I wonder how camels and date palm trees were missing from the whole picture. The personality of the two main characters is very clumsily sketched and many situations are hardly credible. <br /><br />The original idea might have been interesting, but at the end of the day if you are looking for cultural insight, you should skip this movie.


In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.9 MB/s eta 0:00:00


In [9]:
from transformers import BertTokenizer, TFBertForSequenceClassification
#Input preparation

def convert_text_to_feature(review, tokenizer, max_length):  
    return tokenizer.encode_plus(review,
                                add_special_tokens=True,
                                max_length = max_length,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask=True)

def map_feature_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {"input_ids": input_ids,
            "token_type_ids": token_type_ids,
            "attention_mask": attention_masks,}, label

def encode_text(ds, tokenizer, max_length):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
        
    for review, label in ds:
        bert_input = convert_text_to_feature(review[0].numpy().decode(), tokenizer, max_length)
    
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label[0].numpy()])

    return tf.data.Dataset.from_tensor_slices(
                (input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_feature_to_dict)

In [10]:

max_length = 512
learning_rate = 2e-5
epochs = 

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

In [11]:
tokenizer.encode_plus(text_batch[0].numpy().decode(),
                      add_special_tokens = True,
                      max_length = max_length,
                      padding='max_length',
                      truncation=True,
                      return_attention_mask = True)

{'input_ids': [101, 1996, 3185, 2003, 10468, 1037, 11771, 5164, 1997, 10439, 8095, 2075, 18856, 17322, 2015, 2029, 2079, 2025, 3749, 1037, 2613, 2892, 1011, 3451, 12369, 1012, 1996, 2690, 2789, 4190, 1997, 1996, 4990, 2003, 2649, 1999, 1037, 3391, 29348, 2126, 1024, 2045, 5525, 2024, 8494, 5318, 4731, 1010, 6900, 3162, 1999, 1996, 2690, 1997, 1996, 5532, 1010, 2308, 13681, 1999, 2304, 17925, 1998, 7579, 10487, 1012, 1045, 4687, 2129, 19130, 2015, 1998, 3058, 5340, 3628, 2020, 4394, 2013, 1996, 2878, 3861, 1012, 1996, 6180, 1997, 1996, 2048, 2364, 3494, 2003, 2200, 18856, 18163, 6588, 11080, 2098, 1998, 2116, 8146, 2024, 6684, 23411, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 2434, 2801, 2453, 2031, 2042, 5875, 1010, 2021, 2012, 1996, 2203, 1997, 1996, 2154, 2065, 2017, 2024, 2559, 2005, 3451, 12369, 1010, 2017, 2323, 13558, 2023, 3185, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [12]:
batch_size = 32

train_ds = encode_text(train_ds, tokenizer, max_length).shuffle(32).batch(batch_size)
val_ds = encode_text(val_ds, tokenizer, max_length).batch(batch_size)

In [13]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.layers[2].get_weights()

[array([[ 0.02439738,  0.01175153],
        [ 0.02657413, -0.00870533],
        [-0.01170069,  0.01339395],
        ...,
        [-0.01178126, -0.02559171],
        [ 0.01743309, -0.01116544],
        [ 0.00703318,  0.01157149]], dtype=float32),
 array([0., 0.], dtype=float32)]

In [1]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
)

history = model.fit(train_ds, epochs=2, validation_data=val_ds)

NameError: ignored